# Coursera Capstone

This notebook will be mainly used for the capstone project.

### Week 1 Assignment

In [1]:
import pandas as pd
import numpy as np

In [2]:
print("Hello Capstone Project Course!")

Hello Capstone Project Course!


### Week 3 Assignment

#### Instructions

In this assignment, you will be required to explore, segment, and cluster the neighborhoods in the city of Toronto. However, unlike New York, the neighborhood data is not readily available on the internet. What is interesting about the field of data science is that each project can be challenging in its unique way, so you need to learn to be agile and refine the skill to learn new libraries and tools quickly depending on the project.

For the Toronto neighborhood data, a Wikipedia page exists that has all the information we need to explore and cluster the neighborhoods in Toronto. You will be required to scrape the Wikipedia page and wrangle the data, clean it, and then read it into a pandas dataframe so that it is in a structured format like the New York dataset.

Once the data is in a structured format, you can replicate the analysis that we did to the New York City dataset to explore and cluster the neighborhoods in the city of Toronto.

#### Steps

For this assignment, you will be required to explore and cluster the neighborhoods in Toronto.

1. Start by creating a new Notebook for this assignment.


2. Use the Notebook to build the code to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe like the one shown below:

![Table](https://d3c33hcgiwev3.cloudfront.net/imageAssetProxy.v1/7JXaz3NNEeiMwApe4i-fLg_40e690ae0e927abda2d4bde7d94ed133_Screen-Shot-2018-06-18-at-7.17.57-PM.png?expiry=1589068800000&hmac=qzYkoLq5XsqE06VeplzJ13UfdrokXnyfhItpfpO3vps)

3. To create the above dataframe:

    * The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood
    * Only process the cells that have an assigned borough. Ignore cells with a borough that is __Not assigned__.
    * More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that __M5A__ is listed twice and has two neighborhoods: __Harbourfront__ and __Regent Park__. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in __row 11__ in the above table.
    * If a cell has a borough but a __Not assigned__ neighborhood, then the neighborhood will be the same as the borough.
    * Clean your Notebook and add Markdown cells to explain your work and any assumptions you are making.
    * In the last cell of your notebook, use the __.shape__ method to print the number of rows of your dataframe.


4. Submit a link to your Notebook on your Github repository. (__10 marks__)

__Note__: There are different website scraping libraries and packages in Python. For scraping the above table, you can simply use pandas to read the table into a pandas dataframe.

Another way, which would help to learn for more complicated cases of web scraping is using the BeautifulSoup package. Here is the package's main documentation page: http://beautiful-soup-4.readthedocs.io/en/latest/

The package is so popular that there is a plethora of tutorials and examples on how to use it. Here is a very good Youtube video on how to use the BeautifulSoup package: https://www.youtube.com/watch?v=ng2o98k983k

Use pandas, or the BeautifulSoup package, or any other way you are comfortable with to transform the data in the table on the Wikipedia page into the above pandas dataframe.

#### My code

__*1. Using BeautifulSoup to get the table*__

In [3]:
# Cannot get the url via requests using my local installed jupyterlab, WinError 10054 recurring, so comment this cell for lab usage only
# Uncomment below codes if plan to use the BeautifulSoup package

# !conda install -c conda-forge bs4 --yes

from bs4 import BeautifulSoup
import requests

url_wiki = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

html = requests.get(url_wiki).content

soup = BeautifulSoup(html, 'html.parser')

In [4]:
# Uncomment below codes if using the BeautifulSoup package
# Save a version to avoid the potential network errors

# with open ("wiki.txt", "w") as f:
#         f.write(soup.tbody.get_text())
# print("Done!")

In [5]:
pc_ls = []
br_ls = []
nh_ls = []

# Uncomment below if using the BeautifulSoup package 
ls = soup.tbody.get_text().split('\n\n')

# This part need the file "wiki.txt" saved from last cell, use when we cannot use the BeautifulSoup 
# fh = open("wiki.txt","r")
# ls = fh.read().split('\n\n')

for i,item in enumerate(ls):
    item = item.strip()
    if i % 3 == 0: pc_ls.append(item)
    if i % 3 == 1: br_ls.append(item)
    if i % 3 == 2: nh_ls.append(item)

df1 = pd.DataFrame([pc_ls,br_ls,nh_ls]).T
df1.columns = df1.iloc[0,:]
df1 = df1[1:]
df1.reset_index(inplace=True, drop=True)
df1.head()

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,
1,M2A,Not assigned,
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


__*2. Using Pandas to get the table*__

In [6]:
# Got WinError 10054 thus comment this cell

#!conda install -c conda-forge lxml --yes

df2 = pd.read_html(url_wiki)[0]
df2.head()

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,NaN
1,M2A,Not assigned,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


__*3. Process the table*__

We use df1 from part1 for further process.

In [7]:
# Remove lines without Borough values
df = df1[df1['Borough']!='Not assigned']
df.reset_index(inplace=True, drop=True)
df.head()

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [8]:
# Find lines that share the same postal code
df_gp = df.groupby("Postal Code").count()
df_gp[df_gp['Borough']>2]

,Borough,Neighborhood
Postal Code,,


The result indicates the postal codes are already unique in the table.

In [9]:
# Replace NaN in Neighborhood with Borough value
df[df['Neighborhood'].isna()]

,Postal Code,Borough,Neighborhood


The result indicates no such case exists.

In [10]:
df.shape

(103, 3)

----

#### Steps

Now that you have built a dataframe of the postal code of each neighborhood along with the borough name and neighborhood name, in order to utilize the Foursquare location data, we need to get the latitude and the longitude coordinates of each neighborhood.

In an older version of this course, we were leveraging the Google Maps Geocoding API to get the latitude and the longitude coordinates of each neighborhood. However, recently Google started charging for their API: http://geoawesomeness.com/developers-up-in-arms-over-google-maps-api-insane-price-hike/, so we will use the Geocoder Python package instead: https://geocoder.readthedocs.io/index.html.

The problem with this Package is you have to be persistent sometimes in order to get the geographical coordinates of a given postal code. So you can make a call to get the latitude and longitude coordinates of a given postal code and the result would be None, and then make the call again and you would get the coordinates. So, in order to make sure that you get the coordinates for all of our neighborhoods, you can run a while loop for each postal code. Taking postal code __M5G__ as an example, your code would look something like this:


```python
import geocoder # import geocoder

# initialize your variable to None
lat_lng_coords = None

# loop until you get the coordinates
while(lat_lng_coords is None):
  g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
  lat_lng_coords = g.latlng

latitude = lat_lng_coords[0]
longitude = lat_lng_coords[1]
```

Given that this package can be very unreliable, in case you are not able to get the geographical coordinates of the neighborhoods using the Geocoder package, here is a link to a csv file that has the geographical coordinates of each postal code: http://cocl.us/Geospatial_data

Use the Geocoder package or the csv file to create the following dataframe:

![Table](https://d3c33hcgiwev3.cloudfront.net/imageAssetProxy.v1/HZ3jNHNOEeiMwApe4i-fLg_f44f0f10ccfaf42fcbdba9813364e173_Screen-Shot-2018-06-18-at-7.18.16-PM.png?expiry=1589155200000&hmac=X3nmfqHyaEArG_uJjZZ_qpAoA7eVWQ5hnM6XJDmUQxY)

__Important Note__: There is a limit on how many times you can call geocoder.google function. It is 2500 times per day. This should be way more than enough for you to get acquainted with the package and to use it to get the geographical coordinates of the neighborhoods in the Toronto.

Once you are able to create the above dataframe, submit a link to the new Notebook on your Github repository. (__2 marks__)

#### My code

In [11]:
#!conda install -c conda-forge geocoder --yes
import geocoder

In [12]:
# Transform the postal codes into addresses
lc_ls = []
for pc in df['Postal Code']:
    lc_ls.append("{}, Toronto, Ontario".format(pc))

In [13]:
# Get the latitudes and longitudes of the addresses
latlng_dic = {}
for lc in lc_ls:
    latlng = None
    attempt = 0
    while latlng is None:
        g = geocoder.arcgis(lc)
        latlng = g.latlng
        attempt += 1
        if attempt>2:
            break
    latlng_dic[lc] = latlng
    print(lc, latlng)

M3A, Toronto, Ontario [43.75293455500008, -79.33564142299997]
M4A, Toronto, Ontario [43.72810248500008, -79.31188987099995]
M5A, Toronto, Ontario [43.65096410900003, -79.35304116399999]
M6A, Toronto, Ontario [43.723265465000054, -79.45121077799996]
M7A, Toronto, Ontario [43.66179000000005, -79.38938999999993]
M9A, Toronto, Ontario [43.66748067300006, -79.52895286499995]
M1B, Toronto, Ontario [43.80862623100006, -79.18991284599997]
M3B, Toronto, Ontario [43.74890000000005, -79.35721999999998]
M4B, Toronto, Ontario [43.70719267700008, -79.31152927299996]
M5B, Toronto, Ontario [43.65749059800004, -79.37752923699998]
M6B, Toronto, Ontario [43.70727872700007, -79.44750009299997]
M9B, Toronto, Ontario [43.65002250300006, -79.55408903099999]
M1C, Toronto, Ontario [43.78577865700004, -79.15736763799998]
M3C, Toronto, Ontario [43.72214339800007, -79.35202341799999]
M4C, Toronto, Ontario [43.68974004200004, -79.30850701899999]
M5C, Toronto, Ontario [43.65173364700007, -79.37555358799995]
M6C, To

In [14]:
df_latlng = pd.DataFrame(latlng_dic).T
df_latlng.head()

,0,1
"M3A, Toronto, Ontario",43.752935,-79.335641
"M4A, Toronto, Ontario",43.728102,-79.311890
"M5A, Toronto, Ontario",43.650964,-79.353041
"M6A, Toronto, Ontario",43.723265,-79.451211
"M7A, Toronto, Ontario",43.661790,-79.389390


In [15]:
df_latlng.reset_index(drop='True', inplace=True)
df_latlng.columns=['Latitude','Longitude']
df = pd.concat([df,df_latlng],axis=1)
df.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.752935,-79.335641
1,M4A,North York,Victoria Village,43.728102,-79.311890
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.650964,-79.353041
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.723265,-79.451211
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.661790,-79.389390


#### Steps

Explore and cluster the neighborhoods in Toronto. You can decide to work with only boroughs that contain the word Toronto and then replicate the same analysis we did to the New York City data. It is up to you.

Just make sure:

1. to add enough Markdown cells to explain what you decided to do and to report any observations you make.
2. to generate maps to visualize your neighborhoods and how they cluster together.

Once you are happy with your analysis, submit a link to the new Notebook on your Github repository. (__3 marks__)

#### My Code

This part we will group the different locations in Toronto based on the typical venues at that place. Same as the analysis carried out in the lab exercise "Segmenting and Clustering Neighborhoods in New York City". 

First we prepare the necessary data for analysis.

In [16]:
CLIENT_ID = 'XPY14AS3GHQDHDO3HTF1CNG5SOXOWZARINMBXYAU4UOYXIZF'
CLIENT_SECRET = 'YXGDSZG1BMYTEZFHLJL5CQZ0O3F2LMBJWNQGEFPPXSMFDTZO'
VERSION = '20200508'

In [17]:
# Build a function to get the recommended venues of areas correspoding to each posal code

def get_recomm(pc_ls, lat_ls, lng_ls, limit=10, radius=500):
    
    df_venue_full = pd.DataFrame()
    
    for pc, lat, lng in zip(pc_ls, lat_ls, lng_ls):
    
        # Keep track of the progress
        print("Searching for {} ...".format(pc))

        # Get the json from Foursquare
        url_fq = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'\
                                   .format(CLIENT_ID, CLIENT_SECRET, VERSION, lat, lng, radius, limit)
        result = requests.get(url_fq).json()

        # In case some places do not have result
        if len(result['response']['groups'][0]['items'])==0:
            df_venue['Postal Code'] = pc
            continue
        
        # Withdraw the venue info
        df_venue = pd.json_normalize(result['response']['groups'][0]['items'])

        # Only keep necessary info
        df_venue = df_venue[['venue.name', 'venue.location.distance', 'venue.categories']]

        # Withdraw the category info
        df_venue['venue.categories'] = [info[0]['name'] for info in df_venue['venue.categories']] 

        # Formalize the column names
        df_venue.columns = [col.replace('.', ' ').capitalize() for col in df_venue.columns]

        # Add postal codes
        df_venue['Postal Code'] = pc

        # Combine the result
        df_venue_full = pd.concat([df_venue_full,df_venue],axis=0)
    
    df_venue_full.reset_index(drop='True', inplace=True)
    
    return df_venue_full

In [18]:
# Get the result using the location info we got from wiki
df_venue = get_recomm(df['Postal Code'], df['Latitude'], df['Longitude'], limit=100, radius=2000)
df_venue

Searching for M3A ...
Searching for M4A ...
Searching for M5A ...
Searching for M6A ...
Searching for M7A ...
Searching for M9A ...
Searching for M1B ...
Searching for M3B ...
Searching for M4B ...
Searching for M5B ...
Searching for M6B ...
Searching for M9B ...
Searching for M1C ...
Searching for M3C ...
Searching for M4C ...
Searching for M5C ...
Searching for M6C ...
Searching for M9C ...
Searching for M1E ...
Searching for M4E ...
Searching for M5E ...
Searching for M6E ...
Searching for M1G ...
Searching for M4G ...
Searching for M5G ...
Searching for M6G ...
Searching for M1H ...
Searching for M2H ...
Searching for M3H ...
Searching for M4H ...
Searching for M5H ...
Searching for M6H ...
Searching for M1J ...
Searching for M2J ...
Searching for M3J ...
Searching for M4J ...
Searching for M5J ...
Searching for M6J ...
Searching for M1K ...
Searching for M2K ...
Searching for M3K ...
Searching for M4K ...
Searching for M5K ...
Searching for M6K ...
Searching for M1L ...
Searching 

,Venue name,Venue location distance,Venue categories,Postal Code
0,Donalda Golf & Country Club,571,Golf Course,M3A
1,Allwyn's Bakery,1167,Caribbean Restaurant,M3A
2,Galleria Supermarket,1117,Supermarket,M3A
3,Island Foods,1147,Caribbean Restaurant,M3A
4,Brookbanks Park,301,Park,M3A
...,...,...,...,...
8521,Canadian Tire,1829,Hardware Store,M8Z
8522,Subway,971,Sandwich Place,M8Z
8523,Harvey's,1632,Fast Food Restaurant,M8Z
8524,KFC,1632,Fast Food Restaurant,M8Z


In [19]:
# Merge the locaiton info with the venue info
dfmerge = df.merge(df_venue, how='right', on='Postal Code')
dfmerge

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Venue name,Venue location distance,Venue categories
0,M3A,North York,Parkwoods,43.752935,-79.335641,Donalda Golf & Country Club,571,Golf Course
1,M3A,North York,Parkwoods,43.752935,-79.335641,Allwyn's Bakery,1167,Caribbean Restaurant
2,M3A,North York,Parkwoods,43.752935,-79.335641,Galleria Supermarket,1117,Supermarket
3,M3A,North York,Parkwoods,43.752935,-79.335641,Island Foods,1147,Caribbean Restaurant
4,M3A,North York,Parkwoods,43.752935,-79.335641,Brookbanks Park,301,Park
...,...,...,...,...,...,...,...,...
8521,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,...",43.625490,-79.526000,Canadian Tire,1829,Hardware Store
8522,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,...",43.625490,-79.526000,Subway,971,Sandwich Place
8523,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,...",43.625490,-79.526000,Harvey's,1632,Fast Food Restaurant
8524,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,...",43.625490,-79.526000,KFC,1632,Fast Food Restaurant


In [20]:
# Transform into dummies for analysis purpose
df_gd = pd.get_dummies(dfmerge, columns=['Venue categories'], prefix='', prefix_sep='')
df_gd.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Venue name,Venue location distance,Accessories Store,Afghan Restaurant,Airport,...,Warehouse Store,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Xinjiang Restaurant,Yoga Studio,Zoo,Zoo Exhibit
0,M3A,North York,Parkwoods,43.752935,-79.335641,Donalda Golf & Country Club,571,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,M3A,North York,Parkwoods,43.752935,-79.335641,Allwyn's Bakery,1167,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,M3A,North York,Parkwoods,43.752935,-79.335641,Galleria Supermarket,1117,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,M3A,North York,Parkwoods,43.752935,-79.335641,Island Foods,1147,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,M3A,North York,Parkwoods,43.752935,-79.335641,Brookbanks Park,301,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [21]:
# Sum the number of venues in each area
df_vcoun = df_gd.copy()
df_vcoun = df_vcoun.groupby('Postal Code').sum()
df_vcoun = df_vcoun.iloc[:,3:]
df_vcoun.head()

,Accessories Store,Afghan Restaurant,Airport,American Restaurant,Amphitheater,Antique Shop,Arcade,Argentinian Restaurant,Art Gallery,Arts & Crafts Store,...,Warehouse Store,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Xinjiang Restaurant,Yoga Studio,Zoo,Zoo Exhibit
Postal Code,,,,,,,,,,,,,,,,,,,,,
M1B,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2,16
M1C,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
M1E,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
M1G,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
M1H,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,2,0,1,1,0,0


In [22]:
# Get the precentage of each category type in each area
df_vperc = df_vcoun.div(df_vcoun.sum(axis=1), axis='index').round(6)
df_vperc.head()

,Accessories Store,Afghan Restaurant,Airport,American Restaurant,Amphitheater,Antique Shop,Arcade,Argentinian Restaurant,Art Gallery,Arts & Crafts Store,...,Warehouse Store,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Xinjiang Restaurant,Yoga Studio,Zoo,Zoo Exhibit
Postal Code,,,,,,,,,,,,,,,,,,,,,
M1B,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.00,0.042553,0.340426
M1C,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.00,0.000000,0.000000
M1E,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.00,0.000000,0.000000
M1G,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.013514,0.0,...,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.00,0.000000,0.000000
M1H,0.0,0.0,0.0,0.01,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.02,0.0,0.01,0.01,0.000000,0.000000


Now we start to apply the K-Means algorithm.

In [23]:
from sklearn.cluster import KMeans

In [24]:
n_clusters = 8
km = KMeans(n_clusters=n_clusters, init='k-means++', n_init=10, max_iter=300, algorithm='auto')

In [25]:
km.fit(df_vperc)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=8, n_init=10, n_jobs=None, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=0)

In [26]:
df_vcoun['Label']=km.labels_
df_vcoun.head()

,Accessories Store,Afghan Restaurant,Airport,American Restaurant,Amphitheater,Antique Shop,Arcade,Argentinian Restaurant,Art Gallery,Arts & Crafts Store,...,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Xinjiang Restaurant,Yoga Studio,Zoo,Zoo Exhibit,Label
Postal Code,,,,,,,,,,,,,,,,,,,,,
M1B,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2,16,3
M1C,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,7
M1E,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
M1G,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,7
M1H,0,0,0,1,0,0,0,0,0,0,...,0,0,0,2,0,1,1,0,0,7


Based on the fitting result of KMeans, we summarize the venues by labels to get an overview.

In [27]:
df_lab = df_vcoun.groupby("Label").sum()
df_lab.head()

,Accessories Store,Afghan Restaurant,Airport,American Restaurant,Amphitheater,Antique Shop,Arcade,Argentinian Restaurant,Art Gallery,Arts & Crafts Store,...,Warehouse Store,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Xinjiang Restaurant,Yoga Studio,Zoo,Zoo Exhibit
Label,,,,,,,,,,,,,,,,,,,,,
0,0,0,0,2,0,0,0,0,0,2,...,1,0,0,0,0,0,0,0,0,0
1,0,0,0,10,3,1,0,1,0,7,...,1,0,9,0,4,3,0,20,1,0
2,0,1,0,10,0,0,0,0,0,1,...,2,0,0,1,10,2,0,4,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2,16
4,1,0,0,33,1,2,0,0,31,21,...,0,3,6,0,0,2,0,49,0,0


In [28]:
for label in df_lab.index:
    print("---- LABEL {} ----".format(label))
    print(df_lab.iloc[label,:].sort_values(axis=0, ascending=False)[:5])

---- LABEL 0 ----
Pizza Place      40
Coffee Shop      39
Park             29
Bank             23
Grocery Store    22
Name: 0, dtype: uint8
---- LABEL 1 ----
Italian Restaurant    114
Coffee Shop            74
Park                   67
Café                   67
Bakery                 46
Name: 1, dtype: uint8
---- LABEL 2 ----
Coffee Shop            149
Bank                    61
Pharmacy                46
Japanese Restaurant     44
Pizza Place             44
Name: 2, dtype: uint8
---- LABEL 3 ----
Zoo Exhibit             16
Fast Food Restaurant     3
Trail                    2
Restaurant               2
Zoo                      2
Name: 3, dtype: uint8
---- LABEL 4 ----
Coffee Shop           220
Café                  167
Park                  120
Italian Restaurant     86
Bakery                 86
Name: 4, dtype: uint8
---- LABEL 5 ----
Chinese Restaurant    44
Coffee Shop           28
Bakery                15
Bank                  15
Pizza Place           13
Name: 5, dtype: uint8
---- 

Now visualize our result.

In [29]:
df['Label']=km.labels_
df.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Label
0,M3A,North York,Parkwoods,43.752935,-79.335641,3
1,M4A,North York,Victoria Village,43.728102,-79.311890,7
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.650964,-79.353041,0
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.723265,-79.451211,7
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.661790,-79.389390,7


In [30]:
# Define the start point
lat_ini = df["Latitude"].mean()
lng_ini = df["Longitude"].mean()

In [31]:
# Define colors
from matplotlib import cm
from matplotlib import colors

colors_array = cm.gist_rainbow(np.linspace(0, 1, n_clusters))
color_ls = [colors.rgb2hex(i) for i in colors_array]

In [32]:
# Initialize the map
import folium
map = folium.Map(location=[lat_ini, lng_ini], zoom_start=11)

# Mark the clusters
for pc, lat, lng, lab in zip(df['Postal Code'],df['Latitude'],df['Longitude'],df['Label']):
    folium.CircleMarker(location=[lat,lng],  
                        popup="{}, Cluster {}".format(pc,lab),
                        radius=10,
                        stroke=True,
                        color=color_ls[lab],
                        weight=1,
                        opacity=0.4,
                        fill=True,   
                        ).add_to(map)
map

We can see that the algorithm helps us divide areas in Toronto into 8 segments. 

----